In [7]:
import os
import shutil
import pandas as pd
import pyreadstat
import numpy as np

In [8]:
data, meta = pyreadstat.read_sav('Dr.miratashi_3 copy.sav')

In [9]:
def move_folders(folder_list, source_directory, destination_directory):
    for folder_name in folder_list:
        source_path = os.path.join(source_directory, folder_name)
        destination_path = os.path.join(destination_directory, folder_name)
        shutil.move(source_path, destination_path)
        print(f"Moved folder '{folder_name}' from '{source_directory}' to '{destination_directory}'")

In [10]:
def create_pid_folder(parent_directory, pid):
    for folder_name in pid:
        folder_path = os.path.join(parent_directory, str(folder_name))
        os.makedirs(folder_path, exist_ok=True)

        # Create the "FGT" and "BPE" sub folders within each folder
        fgt_folder_path = os.path.join(folder_path, 'FGT')
        bpe_folder_path = os.path.join(folder_path, 'BPE')
        os.makedirs(fgt_folder_path, exist_ok=True)
        os.makedirs(bpe_folder_path, exist_ok=True)

In [11]:
def automated_copy(origin_directory, destination_directory, pid, fgt_idx, bpe_idx):
    img_folder_names = []
    for patient_id in pid:
        dcm_directory = origin_directory + str(patient_id) + '/IMAGE/DCM/'
        for i, (root, dirs, files) in enumerate(os.walk(dcm_directory)):
            if i == 0:
                img_folder_names.append(dcm_directory + str(dirs[i]))
            
    for i, patient_id in enumerate(pid):
        start_fgt, end_fgt = fgt_idx[i]
        start_bpe, end_bpe = bpe_idx[i]
        destination_fgt_folder = destination_directory + str(patient_id) + '/FGT'
        destination_bpe_folder = destination_directory + str(patient_id) + '/BPE'
        for j in range(start_fgt, end_fgt+1):
            if len(str(j)) == 4:
                fgt_img_directory = img_folder_names[i] + '/I000' + str(j)
            else:
                fgt_img_directory = img_folder_names[i] + '/I0000' + str(j)
            shutil.copy(fgt_img_directory, destination_fgt_folder)
            
        for j in range(start_bpe, end_bpe+1):
            if len(str(j)) == 4:
                bpe_img_directory = img_folder_names[i] + '/I000' + str(j)
            else:
                bpe_img_directory = img_folder_names[i] + '/I0000' + str(j)
            shutil.copy(bpe_img_directory, destination_bpe_folder)

In [12]:

def get_folder_names(path, folders):
    for entry in os.scandir(path):
        if entry.is_dir():
            folders.append(entry.name)
    return folders



# Data Preprocessing
## Step 1
### At this step, we get the unique patient IDs and remove data with multiple diagnosis.

In [13]:
folder_names = []

mpath = 'E:/Breast Data/Breast/'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'E:/Breast Data/breast newNAC'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/01/sps'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/02/sps'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/03/breast new'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/03/breast new 2'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/03/nac'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'H:/Breast MRI/03/sps'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'G:/Breast Data/nac'
folder_names = get_folder_names(mpath, folder_names)

mpath = 'G:/Breast Data/sps'
folder_names = get_folder_names(mpath, folder_names)


In [14]:
folder_names = np.array(folder_names)

In [15]:
folder_names_unique = np.unique(folder_names)

In [16]:
data_codes = np.array(data.code)
data_codes_unique = np.unique(data_codes)

In [17]:
same_codes_unique = []
for i in folder_names_unique:
    if i in data_codes_unique:
        same_codes_unique.append(i)
        
same_codes_unique = np.array(same_codes_unique)

In [18]:
not_in_dataset = []
for i in data_codes_unique:
    if i not in same_codes_unique:
        not_in_dataset.append(i)

not_in_dataset_unique = np.unique(np.array(not_in_dataset))

In [19]:
same_codes_unique_mdd_removed = []
for i in same_codes_unique:
    if np.array(data[data.code == i].FG).shape[0] > 1:
        fg_i = np.array(data[data.code == i].FG)
        bpe_i = np.array(data[data.code == i].BPE)
        if np.all(fg_i==fg_i[0]) and np.all(bpe_i==bpe_i[0]):
            same_codes_unique_mdd_removed.append(i)
    else:
        same_codes_unique_mdd_removed.append(i)

In [20]:
not_in_dataset_unique_mdd_removed = []
for i in not_in_dataset_unique:
    if np.array(data[data.code == i].FG).shape[0] > 1:
        fg_i = np.array(data[data.code == i].FG)
        bpe_i = np.array(data[data.code == i].BPE)
        if np.all(fg_i==fg_i[0]) and np.all(bpe_i==bpe_i[0]):
            not_in_dataset_unique_mdd_removed.append(i)
    else:
        not_in_dataset_unique_mdd_removed.append(i)
        

In [21]:
all_ok_data = same_codes_unique_mdd_removed + not_in_dataset_unique_mdd_removed

In [22]:
len(all_ok_data)

1146

# Part 1

In [23]:
mpath = 'H:/Breast MRI/01/sps'
folder_names_part1 = []
folder_names_part1 = get_folder_names(mpath, folder_names_part1)

In [24]:
for name in folder_names_part1:
    if name not in all_ok_data:
        print(name)

In [25]:
no_data_cases = ['9222704', '9999465', '8881092', '11258955', '11843612']

In [26]:
len(all_ok_data)

1146

In [27]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [28]:
len(all_ok_data)

1142

In [29]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/01/sps'

In [159]:
patients_ids_1 = [8860408, 8861603, 8861971, 8866708, 8889836, 8915593, 8915632, 8926218, 8934757, 8935156, 8939476, 8977117,
                  9008081, 9017845, 9108321, 9162757, 9170060, 9173422, 9174037, 9182263, 9221819, 9239385, 9246035, 9354593,
                  9362401, 9363679, 9410575, 9459081, 9459302, 9459663, 9471658, 9471667, 9471756, 9471997, 9474758, 9484852,
                  9484938, 9486031, 9486352, 9492065, 9512135, 9535197, 9535278, 9535434, 9559948, 9571865, 9577964, 9578077,
                  9638156, 9638749, 9646672, 9647040, 9649691, 9658124, 9661700, 9670311, 9670327, 9673120, 9673179, 9673778,
                  9674633, 9680775, 9680867, 9700791, 9701051, 9718836, 9726839, 9727663, 9739752, 9757971, 9773985, 9852342,
                  9900364, 9906687, 9906752, 9910509, 9918129, 9918268, 9918324, 9920466, 9924862, 9926215, 9929361, 9929533,
                  9930243, 9953872, 9953955, 9954110, 9966610, 9970531, 9971081, 9975377, 9975397, 9985427, 9991646, 9994490,
                  9997801, 9997958, 9998057, 10006533, 10007023, 10007991, 10010860, 10018165, 10030277, 10032605, 10032779,
                  10045260, 11201493, 11201533, 11205832, 11208303, 11211696, 11218656, 11224788, 11231020, 11237684, 11239822,
                  11244816, 11247503, 11254075, 11256378, 11264398, 11272908, 11273475, 11286927, 11290544, 11299936, 11301332,
                  11306141, 11331505, 11333648, 11333741, 11338126, 11338192, 11342837, 11344516, 11348359, 11348891, 11351690,
                  11352729, 11355812, 11356851, 11358818, 11366628, 11366836, 11367575, 11369417, 11369686, 11371493, 11372703,
                  11375497, 11377087, 11379700, 11384586, 11385160, 11387992, 11388246, 11389044, 11390408, 11391724, 11393372,
                  11396654, 11572209, 11573971, 11582084, 11582317, 11585576, 11589250, 11840108, 11842798, 11842875, 11846812,
                  11850448, 11851264, 11863836, 11879754, 11881596, 11884718, 11885867, 11887918, 11894552, 11899869, 11900471,
                  11900937, 11903746, 11913796, 11913939, 11915334, 11917076, 11917317]

In [160]:
fgt_1 = [(300, 451), (373, 458), (558, 685), (396, 481), (536, 621), (379, 550), (434, 517), (335, 478), (582, 661), (587, 662),
         (303, 454), (348, 523), (533, 618), (365, 440), (514, 599), (335, 510), (405, 548), (589, 674), (505, 590), (348, 499),
         (521, 606), (387, 542), (407, 492), (554, 639), (365, 450), (365, 450), (432, 513), (490, 567), (371, 442), (464, 549),
         (365, 524), (365, 528), (422, 573), (338, 485), (591, 676), (493, 668), (393, 560), (432, 587), (595, 758), (325, 448),
         (365, 528), (365, 438), (302, 367), (365, 430), (365, 438), (501, 586), (569, 654), (529, 614), (277, 356), (277, 352),
         (281, 366), (249, 314), (490, 575), (420, 505), (277, 352), (536, 679), (277, 352), (421, 506), (474, 559), (439, 524), 
         (441, 526), (471, 560), (315, 406), (449, 536), (421, 510), (466, 551), (332, 499), (269, 354), (483, 518), (479, 650), 
         (461, 546), (435, 520), (552, 735), (431, 518), (443, 530), (423, 586), (402, 545), (485, 628), (380, 523), (430, 515), 
         (496, 671), (597, 800), (385, 464), (451, 540), (299, 384), (313, 404), (315, 402), (317, 404), (435, 520), (448, 639), 
         (612, 771), (439, 526), (517, 602), (500, 585), (405, 490), (555, 640), (426, 513), (297, 382), (277, 350), (506, 649), 
         (464, 549), (454, 539), (455, 598), (471, 556), (435, 598), (407, 554), (426, 577), (299, 384), (467, 662), (421, 612), 
         (376, 551), (258, 343), (275, 450), (429, 620), (337, 512), (500, 675), (350, 525), (396, 575), (406, 581), (405, 490), 
         (380, 555), (413, 604), (393, 568), (417, 592), (383, 558), (423, 508), (404, 579), (417, 640), (461, 604), (402, 577), 
         (286, 461), (359, 534), (587, 762), (399, 574), (385, 560), (420, 607), (370, 545), (417, 502), (405, 490), (395, 570), 
         (552, 727), (363, 538), (567, 718), (369, 544), (407, 582), (415, 602), (393, 568), (426, 601), (385, 560), (389, 564), 
         (344, 519), (355, 530), (428, 619), (561, 756), (397, 572), (670, 857), (402, 577), (356, 519), (412, 591), (366, 541),
         (366, 541), (343, 518), (383, 554), (394, 585), (414, 593), (412, 587), (423, 618), (453, 628), (359, 534), (451, 626),
         (390, 565), (439, 614), (385, 576), (312, 475), (497, 672), (434, 625), (410, 585), (946, 1145), (904, 1095), (868, 1059),
         (347, 522), (314, 489), (387, 562), (395, 570), (348, 523), (396, 587), (380, 559), (588, 803), (301, 460), (593, 772),
         (384, 559)]

In [161]:
bpe_1 = [(1060, 1211), (920, 1005), (2589, 2716), (914, 999), (1162, 1247), (1615, 1786), (1042, 1125), (1235, 1378), (1054, 1133),
         (1035, 1110), (1243, 1394), (1408, 1583), (1071, 1156), (813, 888), (1022, 1107), (1395, 1570), (1349, 1492), (1157, 1242),
         (1061, 1146), (1288, 1439), (1047, 1132), (1323, 1478), (925, 1010), (1080, 1165), (886, 971), (915, 1000), (928, 1009),
         (978, 1055), (820, 891), (980, 1065), (1233, 1392), (1253, 1416), (1252, 1403), (1146, 1293), (1141, 1226), (1441, 1616),
         (1301, 1468), (1280, 1435), (1699, 1862), (1225, 1348), (1253, 1416), (803, 876), (700, 765), (763, 828), (803, 876),
         (1007, 1092), (1107, 1192), (1063, 1148), (769, 848), (749, 824), (803, 888), (671, 736), (1032, 1117), (954, 1039),
         (749, 824), (2267, 2410), (749, 824), (959, 1044), (1012, 1097), (977, 1062), (979, 1064), (1025, 1114), (889, 980), 
         (1002, 1089), (978, 1067), (1015, 1100), (1238, 1405), (783, 868), (1203, 1288), (2382, 2553), (1126, 1211), (969, 1054), 
         (1872, 2055), (984, 1071), (990, 1077), (1423, 1586), (1278, 1421), (1373, 1516), (1256, 1399), (1228, 1313), (1768, 1943), 
         (2001, 2204), (869, 948), (1011, 1100), (849, 934), (880, 971), (868, 955), (871, 958), (957, 1042), (1816, 2007), 
         (2760, 2919), (971, 1058), (1150, 1235), (1155, 1240), (927, 1012), (1227, 1312), (1065, 1152), (834, 919), (754, 827), 
         (1226, 1369), (1161, 1246), (1107, 1192), (1487, 1630), (1108, 1193), (1411, 1574), (1303, 1450), (1342, 1493), (845, 930), 
         (1859, 2054), (1813, 2004), (1648, 1823), (792, 877), (1511, 1686), (1833, 2024), (1579, 1754), (1736, 1911), (1610, 1785), 
         (1716, 1895), (1646, 1821), (936, 1021), (1616, 1791), (1793, 1984), (1635, 1810), (1725, 1900), (1643, 1818), (957, 1042), 
         (1820, 1995), (2723, 2946), (1337, 1480), (1746, 1921), (1522, 1697), (1667, 1842), (1859, 2034), (1671, 1846), (1657, 1832), 
         (1788, 1975), (1610, 1785), (948, 1033), (939, 1024), (1631, 1806), (1788, 1963), (1683, 1858), (1327, 1478), (1605, 1780), 
         (1651, 1826), (1731, 1918), (1635, 1810), (1914, 2089), (1657, 1832), (1673, 1848), (1580, 1755), (1651, 1826), (1820, 2011),
         (1977, 2172), (1653, 1828), (2640, 2827), (1638, 1813), (1520, 1683), (1708, 1887), (1602, 1777), (1602, 1777), (1603, 1778),
         (1595, 1766), (1762, 1953), (1746, 1925), (1684, 1859), (1803, 1998), (1689, 1864), (1667, 1842), (1687, 1862), (1630, 1805),
         (1681, 1856), (1753, 1944), (1464, 1627), (1733, 1908), (1814, 2005), (1652, 1827), (2580, 2779), (2464, 2655), (2408, 2599),
         (1583, 1758), (1370, 1545), (1631, 1806), (1639, 1814), (1590, 1765), (1752, 1943), (2290, 2469), (2842, 3057), (1441, 1600),
         (1861, 2040), (1626, 1801)]

In [33]:
# create_pid_folder(d_directory, patients_ids_1)

In [34]:
# automated_copy(o_directory, d_directory, patients_ids_1, fgt_1, bpe_1)

# Part 2

In [35]:
mpath = 'H:/Breast MRI/02/sps'
folder_names_part2 = []
folder_names_part2 = get_folder_names(mpath, folder_names_part2)

In [36]:
for name in folder_names_part2:
    if name not in all_ok_data:
        print(name)

In [37]:
no_data_cases = ['1191379', '8368349', '8832712', '8145891', '8302127', '8305735', '8414121', '8675439', '8675610', '8682515']

In [38]:
len(all_ok_data)

1142

In [39]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [40]:
len(all_ok_data)

1135

In [41]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [42]:
patients_ids_2 = []

In [43]:
fgt_2 = []

In [44]:
bpe_2 = []

In [45]:
# create_pid_folder(d_directory, patients_ids_2)

In [46]:
# automated_copy(o_directory, d_directory, patients_ids_2, fgt_2, bpe_2)

# Part 3

In [47]:
mpath = 'H:/Breast MRI/03/breast new'
folder_names_part3 = []
folder_names_part3 = get_folder_names(mpath, folder_names_part3)

In [48]:
for name in folder_names_part3:
    if name not in all_ok_data:
        print(name)

In [49]:
no_data_cases = []

In [50]:
len(all_ok_data)

1135

In [51]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [52]:
len(all_ok_data)

1135

In [53]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [54]:
patients_ids_3 = []

In [55]:
fgt_3 = []

In [56]:
bpe_3 = []

In [57]:
# create_pid_folder(d_directory, patients_ids_3)

In [58]:
# automated_copy(o_directory, d_directory, patients_ids_3, fgt_3, bpe_3)

# Part 4

In [59]:
mpath = 'H:/Breast MRI/03/breast new 2'
folder_names_part4 = []
folder_names_part4 = get_folder_names(mpath, folder_names_part4)

In [60]:
folder_list = []
for name in folder_names_part4:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [61]:
len(folder_list)

0

In [62]:
# source_directory = mpath
# destination_directory = "H:/Breast MRI/03/NotOkData/breast new 2"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [63]:
no_data_cases = ['10942269']

In [64]:
len(all_ok_data)

1135

In [65]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [66]:
len(all_ok_data)

1134

In [67]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [68]:
patients_ids_4 = []

In [69]:
fgt_4 = []

In [70]:
bpe_4 = []

In [71]:
# create_pid_folder(d_directory, patients_ids_4)

In [72]:
# automated_copy(o_directory, d_directory, patients_ids_4, fgt_4, bpe_4)

# Part 5

In [73]:
mpath = 'H:/Breast MRI/03/nac'
folder_names_part5 = []
folder_names_part5 = get_folder_names(mpath, folder_names_part5)

In [74]:
folder_list = []
for name in folder_names_part5:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [75]:
len(folder_list)

0

In [76]:
# source_directory = mpath
# destination_directory = "H:/Breast MRI/03/NotOkData/nac"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [77]:
no_data_cases = []

In [78]:
len(all_ok_data)

1134

In [79]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [80]:
len(all_ok_data)

1134

In [81]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [82]:
patients_ids_5 = []

In [83]:
fgt_5 = []

In [84]:
bpe_5 = []

In [85]:
# create_pid_folder(d_directory, patients_ids_5)

In [86]:
# automated_copy(o_directory, d_directory, patients_ids_5, fgt_5, bpe_5)

# Part 6

In [87]:
mpath = 'H:/Breast MRI/03/sps'
folder_names_part6 = []
folder_names_part6 = get_folder_names(mpath, folder_names_part6)

In [88]:
folder_list = []
for name in folder_names_part6:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [89]:
len(folder_list)

0

In [90]:
# source_directory = mpath
# destination_directory = "H:/Breast MRI/03/NotOkData/sps"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [91]:
no_data_cases = ['8390792', '8564813', '8613792', '8756913']

In [92]:
len(all_ok_data)

1134

In [93]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [94]:
len(all_ok_data)

1130

In [95]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [96]:
patients_ids_6 = []

In [97]:
fgt_6 = []

In [98]:
bpe_6 = []

In [99]:
# create_pid_folder(d_directory, patients_ids_1)

In [100]:
# automated_copy(o_directory, d_directory, patients_ids_6, fgt_6, bpe_6)

# Part 7

In [101]:
mpath = "E:/Breast Data/Breast"
folder_names_part7 = []
folder_names_part7 = get_folder_names(mpath, folder_names_part7)

In [102]:
folder_list = []
for name in folder_names_part7:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [103]:
len(folder_list)

0

In [104]:
# source_directory = mpath
# destination_directory = "E:/Breast Data/NotOkData/Breast"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [105]:
no_data_cases = ['11821274']

In [106]:
len(all_ok_data)

1130

In [107]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [108]:
len(all_ok_data)

1129

In [109]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [110]:
patients_ids_7 = []

In [111]:
fgt_7 = []

In [112]:
bpe_7 = []

In [113]:
# create_pid_folder(d_directory, patients_ids_7)

In [114]:
# automated_copy(o_directory, d_directory, patients_ids_7, fgt_7, bpe_7)

# Part 8

In [115]:
mpath = "E:/Breast Data/breast newNAC"
folder_names_part8 = []
folder_names_part8 = get_folder_names(mpath, folder_names_part8)

In [116]:
folder_list = []
for name in folder_names_part8:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [117]:
len(folder_list)

0

In [118]:
# source_directory = mpath
# destination_directory = "E:/Breast Data/NotOkData/breast newNAC"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [119]:
no_data_cases = ['10905568', '10937977']

In [120]:
len(all_ok_data)

1129

In [121]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [122]:
len(all_ok_data)

1127

In [123]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [124]:
patients_ids_8 = []

In [125]:
fgt_8 = []

In [126]:
bpe_8 = []

In [127]:
# create_pid_folder(d_directory, patients_ids_1)

In [128]:
# automated_copy(o_directory, d_directory, patients_ids_1, fgt_1, bpe_1)

# Part 9

In [129]:
mpath = "G:/Breast Data/nac"
folder_names_part9 = []
folder_names_part9 = get_folder_names(mpath, folder_names_part9)

In [130]:
folder_list = []
for name in folder_names_part9:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [131]:
len(folder_list)

0

In [132]:
# source_directory = mpath
# destination_directory = "G:/Breast Data/NotOkData/nac"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [133]:
no_data_cases = []

In [134]:
len(all_ok_data)

1127

In [135]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [136]:
len(all_ok_data)

1127

In [137]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [138]:
patients_ids_9 = []

In [139]:
fgt_9 = []

In [140]:
bpe_9 = []

In [141]:
# create_pid_folder(d_directory, patients_ids_9)

In [142]:
# automated_copy(o_directory, d_directory, patients_ids_9, fgt_9, bpe_9)

# Part 10

In [143]:
mpath = "G:/Breast Data/sps"
folder_names_part10 = []
folder_names_part10 = get_folder_names(mpath, folder_names_part10)

In [144]:
folder_list = []
for name in folder_names_part10:
    if name not in all_ok_data:
        print(name)
        folder_list.append(name)

In [145]:
len(folder_list)

0

In [146]:
# source_directory = mpath
# destination_directory = "G:/Breast Data/NotOkData/sps"  # Replace with the actual destination directory
# move_folders(folder_list, source_directory, destination_directory)

In [147]:
no_data_cases = ['8867592']

In [148]:
len(all_ok_data)

1127

In [149]:
for pid in no_data_cases:
    if pid in all_ok_data:
        all_ok_data.remove(pid)

In [150]:
len(all_ok_data)

1126

In [151]:
# Origin and destination directory
d_directory = 'E:/Selected Data/'
o_directory = 'H:/Breast MRI/02/sps'

In [152]:
patients_ids_10 = []

In [153]:
fgt_10 = []

In [154]:
bpe_10 = []

In [155]:
# create_pid_folder(d_directory, patients_ids_10)

In [156]:
# automated_copy(o_directory, d_directory, patients_ids_10, fgt_10, bpe_10)